# Unified Planning Basic Demo

This python notebook shows the basic usage of the unified planning library.

## Setup the library and the planners

We start by downloading (from github) the unified planning library and the two planners we currently have at our disposal, namely `pyperplan` and `tamer`.

First, we install unified_planning library and its dependencies from PyPi. Here, we use the `--pre` flag to use the latest development build.

In [ ]:
# begin of installation

In [ ]:
!pip install --pre unified-planning

Then, we download and install pyperplan

In [ ]:
!rm -rf up-pyperplan && git clone https://github.com/aiplan4eu/up-pyperplan && pip install up-pyperplan/

Cloning into 'up-pyperplan'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 352 (delta 13), reused 10 (delta 9), pack-reused 270
Receiving objects: 100% (352/352), 64.19 KiB | 1.05 MiB/s, done.
Resolving deltas: 100% (197/197), done.
Processing ./up-pyperplan
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for up-pyperplan: filename=up_pyperplan-0.0.1-py3-none-any.whl size=10799 sha256=b921306ab716d93a0381cf07d6a78343f4740890c9b441477390f04a7bb1e4ad
  Stored in directory: /root/.cache/pip/wheels/dc/a6/7d/1ad6cdeb776400620516a6

We download and install tamer

In [ ]:
!rm -rf up-tamer && git clone https://github.com/aiplan4eu/up-tamer && pip install up-tamer/

Cloning into 'up-tamer'...
remote: Enumerating objects: 407, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 407 (delta 87), reused 74 (delta 74), pack-reused 279
Receiving objects: 100% (407/407), 94.18 KiB | 901.00 KiB/s, done.
Resolving deltas: 100% (247/247), done.
Processing ./up-tamer
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for up-tamer: filename=up_tamer-0.0.1-py3-none-any.whl size=11819 sha256=a0ce8349abf3fa5429f82a5989f103a25657e3ccf1bb185e9351ab8a79afa94a
  Stored in directory: /root/.cache/pip/wheels/4c/7d/54/0e6493c780d4154683fd4e9b06bfbe263

We are now ready to use the Unified-Planning library!

In [ ]:
# end of installation

## Unified-Planning Demo

### Basic imports
The basic imports we need for this demo are abstracted in the `shortcuts` package. Moreover we import the PDDL input/output modules.

In [ ]:
import unified_planning
from unified_planning.shortcuts import *
from unified_planning.io.pddl_writer import PDDLWriter
from unified_planning.io.pddl_reader import PDDLReader

### Problem definition via code

In this example, we will model a very simple robot navigation problem.

#### Types

The first thing to do is to introduce a "UserType" to model the concept of a location. It is possible to introduce as many types as needed; then, for each type we will define a set of objects of that type.  

In addition to `UserType`s we have three built-in types: `Bool`, `Real` and `Integer`. 

In [ ]:
Location = UserType('Location')

#### Fluents and constants

The basic variables of a planning problem are called "fluents" and are quantities that can change over time. Fluents can have differen types, in this first example we will stick to classical "predicates" that are fluents of boolean type. Moreover, fluents can have parameters: effectively describing multiple variables.

For example, a booean fluent `connected` with two parameters of type `Location` (that can be interpreted as `from` and `to`) can be used to model a graph of locations: there exists an edge between two locations `a` and `b` if `connected(a, b)` is true.

In this example, `connected` will be a constant (i.e. it will never change in any execution), but another fluent `robot_at` will be used to model where the robot is: the robot is in locatiopn `l` if and only if `robot_at(l)` is true (we will ensure that exactly one such `l` exists, so that the robot is always in one single location).

In [ ]:
robot_at = unified_planning.model.Fluent('robot_at', BoolType(), l=Location)
connected = unified_planning.model.Fluent('connected', BoolType(), l_from=Location, l_to=Location)

#### Actions

Now we have the problem variables, but in order to describe the possible evolutions of a systems we need to describe how these variables can be changed and how they can evolve. We model this problem using classical, action-based planning, where a set of actions is used to characterize the possible transitions of the system from a state to another.

An action is a transition that can be applied if a specified set of preconditions is satisfied and that prescribes a set of effects that change the value of some fluents. All the fluents that are subjected to the action effects are unchanged.

We allow _lifted_ actions, that are action with parameters: the parameters can be used to specify preconditions or effects and the planner will select among the possible values of each parameters the ones to be used to characterize a specific action. 

In our example, we introduce an action called `move` that has two parameters of type `Location` indicating the current position of the robot `l_from` and the intended destination of the movement `l_to`. The `move(a, b)` action is applicable only when the robot is in position `a` (i.e. `robot_at(a)`) and if `a` and `b` are connected locations (i.e. `connected(a, b)`). As a result of applying the action `move(a, b)`, the robot is no longer in `a` and is instead in location `b`.

In the unified_planning, we can create actions by instantiating the `unified_planning.InstantaneousAction` class; parameters are specified as keyword arguments to the constructor as shown below. Preconditions and effects are added by means of the `add_precondition` and `add_effect` methods. 

In [ ]:
move = unified_planning.model.InstantaneousAction('move', l_from=Location, l_to=Location)
l_from = move.parameter('l_from')
l_to = move.parameter('l_to')
move.add_precondition(connected(l_from, l_to))
move.add_precondition(robot_at(l_from))
move.add_effect(robot_at(l_from), False)
move.add_effect(robot_at(l_to), True)
print(move)

action move(Location l_from, Location l_to) {
    preconditions = [
      connected(l_from, l_to)
      robot_at(l_from)
    ]
    effects = [
      robot_at(l_from) := false
      robot_at(l_to) := true
    ]
    simulated effect = None
  }


#### Creating the problem

The class that represents a planning problem is `unified_planning.Problem`, it contains the set of fluents, the actions, the objects, an intial value for all the fluents and a goal to be reached by the planner. We start by adding the entities we created so far. Note that entities are not bound to one problem, we can create the actions and fluents one and create multiple problems with them.

In [ ]:
problem = unified_planning.model.Problem('robot')
problem.add_fluent(robot_at, default_initial_value=False)
problem.add_fluent(connected, default_initial_value=False)
problem.add_action(move)

The set of objects is a set of `unified_planning.Object` instances, each represnting an element of the domain. In this example, we create `NLOC` (set to 10) locations named `l0` to `l9`. We can create the set of objects and add it to the problem as follows.

In [ ]:
NLOC = 10
locations = [unified_planning.model.Object('l%s' % i, Location) for i in range(NLOC)]
problem.add_objects(locations)

Then, we need to specify the initial state. We used the `default_initial_value` specification when adding the fluents, so it suffices to indicate the fluents that are initially true (this is called "small-world assumption". Without this specification, we would need to initialize all the possible instantiation of all the fluents).

In this example, we connect location `li` with location `li+1`, creating a simple "linear" graph lof locations and we set the initial position of the robot in location `l0`.

In [ ]:
problem.set_initial_value(robot_at(locations[0]), True)
for i in range(NLOC - 1):
    problem.set_initial_value(connected(locations[i], locations[i+1]), True)

Finally, we set the goal of the problem. In this example, we set ourselves to reach location `l9`.

In [ ]:
problem.add_goal(robot_at(locations[-1]))
print(problem)

problem name = robot

types = [Location]

fluents = [
  bool robot_at[l=Location]
  bool connected[l_from=Location, l_to=Location]
]

actions = [
  action move(Location l_from, Location l_to) {
    preconditions = [
      connected(l_from, l_to)
      robot_at(l_from)
    ]
    effects = [
      robot_at(l_from) := false
      robot_at(l_to) := true
    ]
    simulated effect = None
  }
]

objects = [
  Location: [l0, l1, l2, l3, l4, l5, l6, l7, l8, l9]
]

initial fluents default = [
  bool robot_at[l=Location] := false
  bool connected[l_from=Location, l_to=Location] := false
]

initial values = [
  robot_at(l0) := true
  connected(l0, l1) := true
  connected(l1, l2) := true
  connected(l2, l3) := true
  connected(l3, l4) := true
  connected(l4, l5) := true
  connected(l5, l6) := true
  connected(l6, l7) := true
  connected(l7, l8) := true
  connected(l8, l9) := true
]

goals = [
  robot_at(l9)
]




### Solving Planning Problems

The most direct way to solve a planning problem is to select an available planning engine by name and use it to solve the problem. In the following we use `pyperplan` to solve the problem and print the plan.

In [ ]:
with OneshotPlanner(name='pyperplan') as planner:
    result = planner.solve(problem)
    if result.status == up.solvers.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Pyperplan returned: %s" % result.plan)
    else:
        print("No plan found.")

Pyperplan returned: [move(l0, l1), move(l1, l2), move(l2, l3), move(l3, l4), move(l4, l5), move(l5, l6), move(l6, l7), move(l7, l8), move(l8, l9)]


The unified_planning can also automatically select, among the available planners installed on the system, one that is expressive enough for the problem at hand.

In [ ]:
with OneshotPlanner(problem_kind=problem.kind) as planner:
    result = planner.solve(problem)
    print("%s returned: %s" % (planner.name, result.plan))

Tamer returned: [move(l0, l1), move(l1, l2), move(l2, l3), move(l3, l4), move(l4, l5), move(l5, l6), move(l6, l7), move(l7, l8), move(l8, l9)]


In this example, Pyperplan was selected. The `problem.kind` property, returns an object that describes the characteristics of the problem.

In [ ]:
print(problem.kind.features)

{'FLAT_TYPING'}


#### Beyond plan generation

`OneshotPlanner` is not the only operation mode we can invoke from the unified_planning, it is just one way to interact with a planning engine. Another useful functionality is `PlanValidation` that checks if a plan is valid for a problem.

In [ ]:
plan = result.plan
with PlanValidator(problem_kind=problem.kind) as validator:
    if validator.validate(problem, plan):
        print('The plan is valid')
    else:
        print('The plan is invalid')

The plan is valid


It is also possible to use the `Grounding` operation mode to create an equivalent formulation of a problem that does not use parameters for the actions. This openarion mode is implemented by an internal python code, but also some engines offer advanced grounding techniques. 

In [ ]:
with Grounder(problem_kind=problem.kind) as grounder:
    grounding_result = grounder.ground(problem)
    ground_problem = grounding_result.problem
    print(ground_problem)

    # The grounding_result can be used to "lift" a ground plan back to the level of the original problem
    with OneshotPlanner(problem_kind=ground_problem.kind) as planner:
        ground_plan = planner.solve(ground_problem).plan
        print('Ground plan: %s' % ground_plan)
        # Replace the action instances of the grounded plan with their correspoding lifted version
        lifted_plan = ground_plan.replace_action_instances(grounding_result.lift_action_instance)
        print('Lifted plan: %s' % lifted_plan)
        # Test the problem and plan validity
        with PlanValidator(problem_kind=problem.kind) as validator:
            ground_validation = validator.validate(ground_problem, ground_plan)
            lift_validation = validator.validate(problem, lifted_plan)
            Valid = up.solvers.ValidationResultStatus.VALID
            assert ground_validation.status == Valid
            assert lift_validation.status == Valid

problem name = robot

fluents = [
  bool robot_at_l4
  bool robot_at_l2
  bool robot_at_l6
  bool robot_at_l7
  bool robot_at_l1
  bool robot_at_l8
  bool robot_at_l0
  bool robot_at_l5
  bool robot_at_l9
  bool robot_at_l3
]

actions = [
  action move_l5_l6 {
    preconditions = [
      robot_at_l5
    ]
    effects = [
      robot_at_l6 := true
      robot_at_l5 := false
    ]
    simulated effect = None
  }
  action move_l6_l7 {
    preconditions = [
      robot_at_l6
    ]
    effects = [
      robot_at_l7 := true
      robot_at_l6 := false
    ]
    simulated effect = None
  }
  action move_l2_l3 {
    preconditions = [
      robot_at_l2
    ]
    effects = [
      robot_at_l3 := true
      robot_at_l2 := false
    ]
    simulated effect = None
  }
  action move_l0_l1 {
    preconditions = [
      robot_at_l0
    ]
    effects = [
      robot_at_l1 := true
      robot_at_l0 := false
    ]
    simulated effect = None
  }
  action move_l1_l2 {
    preconditions = [
      robot_at_l1

#### Parallel planning

We can invoke different instances of a planner in parallel or different planners and return the first plan that is generated effortlessly.

In [ ]:
with OneshotPlanner(names=['tamer', 'tamer', 'pyperplan'],
                    params=[{'heuristic': 'hadd'}, {'heuristic': 'hmax'}, {}]) as planner:
    plan = planner.solve(problem).plan
    print("%s returned: %s" % (planner.name, plan))

Parallel returned: [move(l0, l1), move(l1, l2), move(l2, l3), move(l3, l4), move(l4, l5), move(l5, l6), move(l6, l7), move(l7, l8), move(l8, l9)]


### PDDL I/O

The library allows to read and write PDDL problems effortlessly.

In [ ]:
w = PDDLWriter(problem)
print(w.get_domain())
print(w.get_problem())

(define (domain robot-domain)
 (:requirements :strips :typing)
 (:types Location)
 (:predicates (robot_at ?l - Location) (connected ?l_from - Location ?l_to - Location))
 (:action move
  :parameters ( ?l_from - Location ?l_to - Location)
  :precondition (and (connected ?l_from ?l_to) (robot_at ?l_from))
  :effect (and (not (robot_at ?l_from)) (robot_at ?l_to)))
)

(define (problem robot-problem)
 (:domain robot-domain)
 (:objects 
   l0 l1 l2 l3 l4 l5 l6 l7 l8 l9 - Location
 )
 (:init (robot_at l0) (connected l0 l1) (connected l1 l2) (connected l2 l3) (connected l3 l4) (connected l4 l5) (connected l5 l6) (connected l6 l7) (connected l7 l8) (connected l8 l9))
 (:goal (and (robot_at l9)))
)



In [ ]:
!wget https://raw.githubusercontent.com/aiplan4eu/unified-planning/master/unified_planning/test/pddl/depot/domain.pddl -O /tmp/depot_domain.pddl

--2022-04-22 10:20:25--  https://raw.githubusercontent.com/aiplan4eu/unified-planning/master/unified_planning/test/pddl/depot/domain.pddl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1377 (1.3K) [text/plain]
Saving to: ‘/tmp/depot_domain.pddl’

/tmp/depot_domain.p 100%[===================>]   1.34K  --.-KB/s    in 0s      

2022-04-22 10:20:26 (48.6 MB/s) - ‘/tmp/depot_domain.pddl’ saved [1377/1377]



In [ ]:
!wget https://raw.githubusercontent.com/aiplan4eu/unified-planning/master/unified_planning/test/pddl/depot/problem.pddl -O /tmp/depot_problem.pddl

--2022-04-22 10:20:26--  https://raw.githubusercontent.com/aiplan4eu/unified-planning/master/unified_planning/test/pddl/depot/problem.pddl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948 [text/plain]
Saving to: ‘/tmp/depot_problem.pddl’

/tmp/depot_problem. 100%[===================>]     948  --.-KB/s    in 0s      

2022-04-22 10:20:26 (50.0 MB/s) - ‘/tmp/depot_problem.pddl’ saved [948/948]



In [ ]:
reader = PDDLReader()
pddl_problem = reader.parse_problem('/tmp/depot_domain.pddl', '/tmp/depot_problem.pddl')
print(pddl_problem)

problem name = depotprob1818

types = [object]

fluents = [
  bool at[x=object, y=object]
  bool on[x=object, y=object]
  bool in[x=object, y=object]
  bool lifting[x=object, y=object]
  bool available[x=object]
  bool clear[x=object]
  bool place[x=object]
  bool locatable[x=object]
  bool depot[x=object]
  bool distributor[x=object]
  bool truck[x=object]
  bool hoist[x=object]
  bool surface[x=object]
  bool pallet[x=object]
  bool crate[x=object]
]

actions = [
  action drive(object x, object y, object z) {
    preconditions = [
      (truck(x) and place(y) and place(z) and at(x, y))
    ]
    effects = [
      at(x, z) := true
      at(x, y) := false
    ]
    simulated effect = None
  }
  action lift(object x, object y, object z, object p) {
    preconditions = [
      (hoist(x) and crate(y) and surface(z) and place(p) and at(x, p) and available(x) and at(y, p) and on(y, z) and clear(y))
    ]
    effects = [
      lifting(x, y) := true
      clear(z) := true
      at(y, p) := fa

A parsed PDDL problem is just a normal problem that can be solved.

In [ ]:
print(pddl_problem.kind.features)
with OneshotPlanner(name='pyperplan') as planner:
    result = planner.solve(pddl_problem)
    print("%s returned: %s" % (planner.name, result.plan))


{'FLAT_TYPING'}
Pyperplan returned: [lift(hoist1, crate0, pallet1, distributor0), lift(hoist0, crate1, pallet0, depot0), load(hoist0, crate1, truck1, depot0), drive(truck1, depot0, distributor0), load(hoist1, crate0, truck1, distributor0), unload(hoist1, crate1, truck1, distributor0), drive(truck1, distributor0, distributor1), drop(hoist1, crate1, pallet1, distributor0), unload(hoist2, crate0, truck1, distributor1), drop(hoist2, crate0, pallet2, distributor1)]
